In [1]:
import pandas as pd

In [64]:
df_train = pd.read_csv('C:/Users/katya/Fake_News/data_train.csv')
df_test = pd.read_csv('C:/Users/katya/Fake_News/data_test.csv')

In [65]:
frames = [df_train, df_test]

#concatenate dataframes
df = pd.DataFrame()
df = pd.concat(frames)

In [70]:
df.reset_index()

,index,Unnamed: 0,title,text,subject,date,class,source
0,0,0,"Republicans unveil tax cut bill, but the hard ...",U.S. House of Representatives Republicans un...,politicsNews,"November 2, 2017",1,WASHINGTON
1,1,1,"German minister says EU, NATO nations concerne...",Foreign ministers from NATO and European Uni...,politicsNews,"January 16, 2017",1,BRUSSELS
2,2,2,'The Wire' actor Pierce arrested after scuffle...,"Actor Wendell Pierce, who starred in HBO’s “...",politicsNews,"May 16, 2016",1,ATLANTA
3,3,3,Evangelical Jerry Falwell Jr. to head Trump ed...,Evangelical Christian leader Jerry Falwell J...,politicsNews,"February 1, 2017",1,NaN
4,4,4,Fed official says U.S. immigration crackdown c...,The Trump administration’s crackdown on ille...,politicsNews,"May 31, 2017",1,NEW
...,...,...,...,...,...,...,...,...
44262,10445,10445,IS HILLARY’S CAMPAIGN In YUGE Trouble? SHOCKIN...,Meanwhile Trump is about to make history for c...,politics,"Apr 30, 2016",0,NaN
44263,10446,10446,"Just Days In, Trump Administration Ready To D...",Brand new White House Press Secretary Sean Spi...,News,"January 24, 2017",0,NaN
44264,10447,10447,"HIV POSITIVE, CRACK SMOKING, Liberal Actor Cha...","Less than one year ago, Charlie Sheen s ex-wif...",politics,"Dec 29, 2016",0,NaN
44265,10448,10448,OMINOUS WARNING TO EUROPEANS: You Are Committi...,"Douglas Murray, author of The Strange Death Of...",left-news,"Jul 3, 2017",0,NaN


In [50]:
len(df)

44267

In [55]:
df=df.dropna()

In [71]:
## get independent features
x= df.drop('class',axis=1)

df['class'] = df['class'].replace({'T': 1})
df['class'] = df['class'].replace({'F': 0})

y=df['class']

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [72]:
df['class']

0        1
1        1
2        1
3        1
4        1
        ..
10445    0
10446    0
10447    0
10448    0
10449    0
Name: class, Length: 44267, dtype: int64

In [23]:
import tensorflow as tf
tf.__version__


'2.6.0'

In [24]:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

# vocabularry size
voc_size=5000

In [73]:
messages= x.copy()
messages.reset_index(inplace=True)

import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [75]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr)

[[4606, 111, 4079, 1715, 3117, 4966, 2024, 2914], [1043, 2388, 2542, 920, 1210, 3337, 4520, 2904, 1210, 72], [1897, 4395, 1061, 4972, 2872, 915, 3383], [4155, 1578, 4958, 3790, 3856, 2904, 3508, 2363, 879], [2656, 1308, 2542, 3848, 2957, 2969, 3091, 932, 4262], [2461, 2904, 3262, 1850, 4159, 2288, 3747, 537, 3290], [2904, 3319, 4052, 1376, 27, 3848, 178, 3117], [598, 3046, 580, 1379, 1180, 1012, 1860], [3848, 1393, 1470, 3062, 52, 2206, 4079, 4092, 1063, 3454], [2904, 3321, 4446, 4707, 670, 406, 2298, 1471, 4251], [991, 1977, 4793, 3208, 4907, 2522, 3117, 2343], [858, 3939, 2116, 1607, 628, 3490, 105, 4251], [4907, 2522, 2897, 4098, 2810, 668, 1860, 60], [51, 4131, 2542, 341, 3274, 1177, 3848], [550, 158, 3984, 1250, 668, 2198, 2753], [3926, 4205, 341, 3501, 72, 3614, 4510, 4911, 3602], [2904, 2434, 3739, 3201, 3735], [4092, 1252, 4756, 2969, 3383, 819], [2461, 920, 4846, 3239, 1420, 477, 3452], [4606, 4272, 4947, 2904, 1034, 1677, 721], [2904, 1328, 803, 3848, 4698, 3423, 3076, 1730, 

In [76]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4966 2024 2914]
 [   0    0    0 ... 2904 1210   72]
 [   0    0    0 ... 2872  915 3383]
 ...
 [   0    0    0 ... 1533 4977 1762]
 [   0    0    0 ... 2394 2614 2295]
 [   0    0    0 ... 1441 4533 2314]]


In [77]:
len(embedded_docs)


44267

In [78]:
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [80]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [81]:
y_train

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [82]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
464/464 [==============================] - 34s 68ms/step - loss: 0.2379 - accuracy: 0.8988 - val_loss: 0.1564 - val_accuracy: 0.9367
Epoch 2/10
464/464 [==============================] - 26s 57ms/step - loss: 0.1291 - accuracy: 0.9494 - val_loss: 0.1462 - val_accuracy: 0.9428
Epoch 3/10
464/464 [==============================] - 17s 37ms/step - loss: 0.0984 - accuracy: 0.9620 - val_loss: 0.1410 - val_accuracy: 0.9456
Epoch 4/10
464/464 [==============================] - 26s 56ms/step - loss: 0.0745 - accuracy: 0.9724 - val_loss: 0.1609 - val_accuracy: 0.9400
Epoch 5/10
464/464 [==============================] - 30s 65ms/step - loss: 0.0577 - accuracy: 0.9778 - val_loss: 0.1740 - val_accuracy: 0.9443
Epoch 6/10
464/464 [==============================] - 29s 62ms/step - loss: 0.0439 - accuracy: 0.9843 - val_loss: 0.2011 - val_accuracy: 0.9416
Epoch 7/10
464/464 [==============================] - 27s 58ms/step - loss: 0.0326 - accuracy: 0.9883 - val_loss: 0.2306 - val_accuracy:

In [83]:
y_pred= model.predict_classes(x_test)
y_pred

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
ac= accuracy_score(y_test,y_pred)
print(ac)

In [ ]:
from tensorflow.keras.layers import Dropout
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(Dropout(0.3))
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred= model.predict_classes(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cmm= confusion_matrix(y_test,y_pred)
print(cmm)

In [ ]:
ac= accuracy_score(y_test,y_pred)
print(ac)